In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# -- coding: utf-8 --

import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
from nltk.stem.isri import ISRIStemmer
import nltk
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn import cluster
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans



def ROUGE2_pre(g, h):

    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    pre_rouge_2 = count / len(pro_words)

    return pre_rouge_2



def ROUGE2_re(g, h):


    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    # Storing the final accuracy of Rouge-1
    if(len(ac_words)==0):
      re_rouge_2 = count / 1
    else:
      re_rouge_2 = count / len(ac_words)

    return re_rouge_2




def ROUGE1_pre(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break


    pre_rouge_1 = count / len(pro_words)

    return pre_rouge_1


def ROUGE1_re(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    if(len(ac_words)==0):
      re_rouge_1 = count / 1
    else:
      re_rouge_1 = count / len(ac_words)

    return re_rouge_1




def word2vecsummary(text):

    t_model = KeyedVectors.load('model_urdu256-5.model')

    text = text.replace('\r\n', ' ')


    sentences_tokens = text.split('۔')


    senCount = 0;
    sentenceVec = {}
    tsne = TSNE(n_components=2)
    for sen in sentences_tokens:
        sen = [word for word in nltk.word_tokenize(sen) if word in t_model]
        if len(sen) >= 1:
            sentenceVec[senCount] = np.mean(t_model[sen], axis=0)
            senCount += 1

    NUM_CLUSTERS = 6 # num represents number of sentences that should be produced
    kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
    newSenVec = []
    for i in sentenceVec:
        newSenVec.append(sentenceVec[i])
# =======================
# =======================

    n_clusters = NUM_CLUSTERS
# Create the KMEANS
    kmeans2 = KMeans(n_clusters=n_clusters, random_state=0)
# =======================
# Fit the sentences to each cluster
# =======================
    kmeans2 = kmeans2.fit(newSenVec)
    avg2 = []

    for j in range(n_clusters):
        idx = np.where(kmeans2.labels_ == j)[0]
        avg2.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans2.cluster_centers_, newSenVec)
    ordering = sorted(range(n_clusters), key=lambda k: avg2[k])
    summary2 = '۔'.join([sentences_tokens[closest[idx]] for idx in ordering])

# =======================
# Fit the sentences to each cluster
# =======================
    kmeans.fit(newSenVec)
    labels = kmeans.labels_
    n_clusters1 = n_clusters
    avg1 = []
    for j in range(n_clusters1):
        idx1 = np.where(kmeans.labels_ == j)[0]
        avg1.append(np.mean(idx1))

    appearedLabesl = []
    index = 0
    for i in labels:
        appearedLabesl.append(i)
        index = index + 1
    return summary2









df = pd.read_csv('Urdu_7_Books.csv')


X = df.Text
y = df.Khulasay
z = df.Markazi_Khayal

j = 0
precount = 1
presumscore = 0
preavg = 0

resumscore = 0
reavg = 0

f_score = 0
f_sums = 0
f_avg = 0

presumscore2 = 0
preavg2 = 0

resumscore2 = 0
reavg2 = 0

f_score2 = 0
f_sums2 = 0
f_avg2 = 0


size = 6

for x in X:
    XC = word2vecsummary(str(' '+X[j]+' '))
    precision = ROUGE1_pre(XC, y[j])
    recall = ROUGE1_re(XC, y[j])

    presumscore = presumscore + precision
    resumscore = resumscore + recall

    preavg = presumscore / precount
    reavg = resumscore / precount

    # ==========ROUGE-2==========================================

    precision2 = ROUGE2_pre(XC, y[j])
    recall2 = ROUGE2_re(XC, y[j])
    # print(XC)

    presumscore2 = presumscore2 + precision2
    resumscore2 = resumscore2 + recall2

    preavg2 = presumscore2 / precount
    reavg2 = resumscore2 / precount

    precount = precount + 1
    j = j + 1

f_score = 2 * (preavg * reavg) / (preavg + reavg)

f_score2 = 2 * (preavg2 * reavg2) / (preavg2 + reavg2)

print("average recall is: ")
print(reavg)

print("average Precision is: ")
print(preavg)

print("f-measure average: ")
print(f_score)

print("for ROUGE 2")

print("ROUGE-2 average Precision is: ")
print(preavg2)

print("AROUGE-2 average recall is: ")
print(reavg2)

print("ROUGE-2 f-measure average: ")
print(f_score2)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

average recall is: 
0.7018936035262976
average Precision is: 
0.2374801525231554
f-measure average: 
0.3548870701294687
for ROUGE 2
ROUGE-2 average Precision is: 
0.09843618750130571
AROUGE-2 average recall is: 
0.3125576876651052
ROUGE-2 f-measure average: 
0.14971993018396845
